In [15]:
import numpy as np
import sklearn 
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

In [12]:
nltk.download('punkt')
nltk.download('popular')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\motve\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\motve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\motve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\motve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\motve\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\motve\AppDa

True

In [22]:
text = 'Биба в бобе, боба в бабе. Я люблю кинчики, акции очень сильно упали '
word_tokens = nltk.word_tokenize(text)
stop_words = set(stopwords.words('russian'))
filtered_tokens = [word for word in word_tokens if word not in stop_words]


In [21]:
stemmer = SnowballStemmer('russian')
tokens = word_tokenize(text)
stemmed_words = [stemmer.stem(word) for word in tokens]
stemmed_words

['биб',
 'в',
 'боб',
 ',',
 'боб',
 'в',
 'баб',
 '.',
 'я',
 'любл',
 'кинчик',
 ',',
 'акц',
 'очен',
 'сильн',
 'упа']